<a href="https://colab.research.google.com/github/AlexSandroJr99/Gabarito/blob/main/GabaritoLeitura.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install pdf2image
!apt-get install poppler-utils

In [ ]:
from pdf2image import convert_from_path
import os
import time

def convert_pdf_to_jpeg(pdf_file, output_dir, doc_id, dpi=100):
    """Converte um PDF para imagens JPEG."""
    images = convert_from_path(pdf_file, dpi=dpi)

    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for i, image in enumerate(images):
        image.save(os.path.join(output_dir, f'Gabarito_{doc_id}_page_{i+1}.jpeg'), 'JPEG')

def process_new_pdfs(input_dir, output_dir, processed_files_file="processed_files.txt", initial_doc_id=1, dpi=100):
    """Monitora o diretório de entrada e processa os novos PDFs."""


    if os.path.exists(processed_files_file):
        with open(processed_files_file, "r") as f:
            processed_files = set(line.strip().split('|')[0] for line in f)
    else:
        processed_files = set()

    doc_id_counter = initial_doc_id

    while True:
        pdf_files = [f for f in os.listdir(input_dir) if f.endswith('.pdf') and f not in processed_files]

        if not pdf_files:
            print("Todos os PDFs foram processados. Encerrando...")
            break

        for pdf_file in pdf_files:
            pdf_path = os.path.join(input_dir, pdf_file)


            convert_pdf_to_jpeg(pdf_path, output_dir, doc_id_counter, dpi)


            processed_files.add(pdf_file)

            with open(processed_files_file, "a") as f:
                f.write(f"{pdf_file}|Gabarito_{doc_id_counter}\n")

            doc_id_counter += 1
            print(f"Processado: {pdf_file}")

        time.sleep(5)

input_dir = '/content/PDF'
output_dir = '/content/images_JPEG'

process_new_pdfs(input_dir, output_dir)


In [ ]:
import cv2
import os
import numpy as np

def salvar_recorte(imagem, caminho, nome_arquivo):
    if not os.path.exists(caminho):
        os.makedirs(caminho)
    cv2.imwrite(os.path.join(caminho, nome_arquivo), imagem)

largura_alvo = 76
altura_alvo = 32
tolerancia = 60
deslocamento = 40
caminho_salvar = "/content/Recortes"
caminho_imagens = "/content/images_JPEG"


arquivos = sorted([f for f in os.listdir(caminho_imagens) if f.endswith(".jpeg")])

for arquivo in arquivos:
    gabarito_id = os.path.splitext(arquivo)[0]
    imagem = cv2.imread(os.path.join(caminho_imagens, arquivo), cv2.IMREAD_GRAYSCALE)

    if imagem is None:
        print(f"Erro ao carregar a imagem {arquivo}.")
        continue


    imagem_escala_alta = cv2.resize(imagem, None, fx=3, fy=3, interpolation=cv2.INTER_CUBIC)
    imagem_blur = cv2.medianBlur(imagem_escala_alta, 7)
    _, imagem_binaria = cv2.threshold(imagem_blur, 160, 255, cv2.THRESH_BINARY_INV)
    contornos, _ = cv2.findContours(imagem_binaria, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    retangulos = []
    for contorno in contornos:
        epsilon = 0.03 * cv2.arcLength(contorno, True)
        approx = cv2.approxPolyDP(contorno, epsilon, True)

        if 4 <= len(approx) <= 6:
            x, y, w, h = cv2.boundingRect(contorno)
            if (abs(w - largura_alvo) <= tolerancia) and (abs(h - altura_alvo) <= tolerancia):
                retangulos.append((x, y, w, h))

    if len(retangulos) > 8:
        retangulos.sort(key=lambda r: abs(r[2] - largura_alvo) + abs(r[3] - altura_alvo))
        retangulos = retangulos[:8]

    retangulos.sort(key=lambda r: (r[1], r[0]))

    if len(retangulos) >= 4:
        x_min = min([r[0] for r in retangulos[:4]])
        y_min = min([r[1] for r in retangulos[:4]])
        x_max = max([r[0] + r[2] for r in retangulos[:4]])
        y_max = max([r[1] + r[3] for r in retangulos[:4]])

        cropped_matricula = imagem_escala_alta[y_min + deslocamento:y_max - deslocamento, x_min + deslocamento:x_max - deslocamento]
        cropped_matricula = cv2.resize(cropped_matricula, (int(cropped_matricula.shape[1] / 3), int(cropped_matricula.shape[0] / 3)))
        salvar_recorte(cropped_matricula, caminho_salvar, f"{gabarito_id}_matricula.jpeg")

    if len(retangulos) >= 8:
        x_min_5_8 = min([r[0] for r in retangulos[4:8]])
        y_min_5_8 = min([r[1] for r in retangulos[4:8]])
        x_max_5_8 = max([r[0] + r[2] for r in retangulos[4:8]])
        y_max_5_8 = max([r[1] + r[3] for r in retangulos[4:8]])

        cropped_alternativas = imagem_escala_alta[y_min_5_8 + deslocamento:y_max_5_8 - deslocamento, x_min_5_8 + deslocamento:x_max_5_8 - deslocamento]
        cropped_alternativas = cv2.resize(cropped_alternativas, (int(cropped_alternativas.shape[1] / 3), int(cropped_alternativas.shape[0] / 3)))
        salvar_recorte(cropped_alternativas, caminho_salvar, f"{gabarito_id}_alternativas.jpeg")

In [ ]:
import cv2
import os

def salvar_recorte(imagem, caminho, nome_arquivo):
    if not os.path.exists(caminho):
        os.makedirs(caminho)
    cv2.imwrite(os.path.join(caminho, nome_arquivo), imagem)


input_dir = '/content/Recortes'
output_dir = '/content/Colunas'


for arquivo in os.listdir(input_dir):
    if arquivo.endswith('.jpeg') and "_alternativas" in arquivo:
        gabarito_id = os.path.splitext(arquivo)[0]
        imagem_path = os.path.join(input_dir, arquivo)
        imagem = cv2.imread(imagem_path)

        if imagem is None:
            print(f"Erro ao carregar a imagem: {imagem_path}")
            continue

        margens_esquerda = [22, 3, 4, 6]
        larguras = [140, 140, 140, 140]
        distancias = [26, 17, 17, 30]
        largura_acumulada = 0

        for i in range(4):
            x1 = largura_acumulada + margens_esquerda[i]
            x2 = x1 + larguras[i]
            coluna_cortada = imagem[:, x1:x2]
            salvar_recorte(coluna_cortada, output_dir, f'{gabarito_id}_Coluna_{i+1}.jpeg')
            largura_acumulada = x2 + distancias[i] if i < 3 else x2

In [ ]:
import cv2
import numpy as np
import pandas as pd
import os

def Detectar(imagem, caminho, nome_arquivo):
    if not os.path.exists(caminho):
        os.makedirs(caminho)
    cv2.imwrite(os.path.join(caminho, nome_arquivo), imagem)

def converter_para_300dpi(imagem):
    return cv2.resize(imagem, None, fx=3, fy=3, interpolation=cv2.INTER_CUBIC)

def processar_matricula(matricula_path, gabarito_id):
    imagem = cv2.imread(matricula_path, cv2.IMREAD_GRAYSCALE)
    if imagem is None:
        raise FileNotFoundError(f"Imagem não encontrada: {matricula_path}")

    imagem_blur = cv2.medianBlur(imagem, 7)
    circulos = cv2.HoughCircles(imagem_blur, cv2.HOUGH_GRADIENT, dp=1.2, minDist=20,
                                 param1=47, param2=30, minRadius=5, maxRadius=40)

    if circulos is not None:
        circulos = np.uint16(np.around(circulos))
        for circulo in circulos[0, :]:
            x, y = circulo[0], circulo[1]
            cv2.circle(imagem, (x, y), circulo[2], (0, 255, 0), 2)

    colunas = [str(i) for i in range(1, 9)]
    linhas = list(range(10))
    df = pd.DataFrame(index=linhas, columns=colunas)
    df.fillna('', inplace=True)

    if circulos is not None:
        circulos = np.uint16(np.around(circulos))
        for circulo in circulos[0, :]:
            x, y = circulo[0], circulo[1]
            linha = y // (imagem.shape[0] // 10)
            coluna = x // (imagem.shape[1] // 8)

            if linha < len(linhas) and coluna < len(colunas):
                df.loc[linha, colunas[coluna]] = str(linha)

    lista_numeros_detectados = []
    for coluna in colunas:
        coluna_sem_nan = df[coluna].replace('', np.nan).dropna()
        if not coluna_sem_nan.empty:
            lista_numeros_detectados.append(coluna_sem_nan.iloc[0])

    numero_matricula = ''.join(map(str, lista_numeros_detectados)) if lista_numeros_detectados else ''
    return numero_matricula

def detectar_bolinha(imagem):
    minRadius = 10
    maxRadius = 50
    imagem_blur = cv2.medianBlur(imagem, 11)
    _, imagem_binaria = cv2.threshold(imagem_blur, 160, 255, cv2.THRESH_BINARY_INV)
    circulos = cv2.HoughCircles(imagem_blur, cv2.HOUGH_GRADIENT, dp=1.1, minDist=20,
                                 param1=50, param2=35, minRadius=minRadius, maxRadius=maxRadius)

    bolinhas_detectadas = []
    if circulos is not None:
        circulos = np.round(circulos[0, :]).astype("int")
        for (x, y, r) in circulos:
            mascara = np.zeros_like(imagem_binaria)
            cv2.circle(mascara, (x, y), r, 255, -1)
            area_total = cv2.countNonZero(mascara)
            area_preenchida = cv2.countNonZero(imagem_binaria & mascara)
            if area_preenchida / area_total > 0.5:
                cv2.circle(imagem, (x, y), r, (0, 255, 0), 2)
                cv2.circle(imagem, (x, y), 2, (0, 0, 255), 3)
                bolinhas_detectadas.append((x, y))

    return bolinhas_detectadas

def processar_circulos(bolinhas, imagem_shape, inicio_questao, fim_questao):
    colunas = ['A', 'B', 'C', 'D', 'E']
    linhas = list(range(inicio_questao, fim_questao + 1))
    df = pd.DataFrame(index=linhas, columns=colunas)
    df.fillna(0, inplace=True)

    for (x, y) in bolinhas:
        linha = y // (imagem_shape[0] // (fim_questao - inicio_questao + 1)) + inicio_questao
        coluna = x // (imagem_shape[1] // 5)

        if linha <= fim_questao and coluna < len(colunas):
            df.loc[linha, colunas[coluna]] = 1

    return df

def processar_alternativas(recortes):
    df_geral = pd.DataFrame(columns=['MATRICULA', 'QUESTAO', 'A', 'B', 'C', 'D', 'E'])

    for recorte_path, inicio_questao, fim_questao in recortes:
        imagem = cv2.imread(recorte_path, cv2.IMREAD_GRAYSCALE)
        if imagem is None:
            raise FileNotFoundError(f"Imagem não encontrada: {recorte_path}")

        imagem = converter_para_300dpi(imagem)
        bolinhas = detectar_bolinha(imagem)
        df_parcial = processar_circulos(bolinhas, imagem.shape, inicio_questao, fim_questao)
        df_parcial.reset_index(inplace=True)
        df_parcial.rename(columns={'index': 'QUESTAO'}, inplace=True)

        for questao in range(inicio_questao, fim_questao + 1):
            if questao <= 60:
                df_parcial.loc[df_parcial['QUESTAO'] == questao, 'QUESTAO'] = questao

        df_geral = pd.concat([df_geral, df_parcial], ignore_index=True)

    return df_geral

def salvar_csv_combinado(matricula_paths, recortes, gabaritos_ids):
    df_geral = pd.DataFrame(columns=['Id_Gabarito', 'MATRICULA', 'QUESTAO', 'A', 'B', 'C', 'D', 'E'])

    for gabarito_id, matricula_path in zip(gabaritos_ids, matricula_paths):
        numero_matricula = processar_matricula(matricula_path, gabarito_id)
        recortes_atual = [
            (f'/content/Colunas/{gabarito_id}_alternativas_Coluna_1.jpeg', 1, 15),
            (f'/content/Colunas/{gabarito_id}_alternativas_Coluna_2.jpeg', 16, 30),
            (f'/content/Colunas/{gabarito_id}_alternativas_Coluna_3.jpeg', 31, 45),
            (f'/content/Colunas/{gabarito_id}_alternativas_Coluna_4.jpeg', 46, 60)
        ]
        df_alternativas = processar_alternativas(recortes_atual)
        df_alternativas['Id_Gabarito'] = gabarito_id
        df_alternativas['MATRICULA'] = numero_matricula
        df_geral = pd.concat([df_geral, df_alternativas], ignore_index=True)

    csv_file_path = '/content/matricula_e_respostas.csv'
    df_geral.to_csv(csv_file_path, sep=';', index=False)

def automatizar_gabaritos_e_matriculas(pasta_recortes):
    arquivos = os.listdir(pasta_recortes)
    gabaritos_ids = [arquivo.replace('_alternativas.jpeg', '') for arquivo in arquivos if 'alternativas' in arquivo]
    matricula_paths = [os.path.join(pasta_recortes, f'{gabarito_id}_matricula.jpeg') for gabarito_id in gabaritos_ids]
    matricula_paths = [path for path in matricula_paths if os.path.exists(path)]
    gabaritos_ids = [gabarito_id for gabarito_id in gabaritos_ids if os.path.exists(os.path.join(pasta_recortes, f'{gabarito_id}_matricula.jpeg'))]

    if not matricula_paths:
        print("Nenhum arquivo de matrícula encontrado!")
        return

    salvar_csv_combinado(matricula_paths, [], gabaritos_ids)

automatizar_gabaritos_e_matriculas('/content/Recortes')

In [ ]:
import pandas as pd
import json
import hashlib
import os

def obter_nome_arquivo_automatico(caminho_processed_files, pasta_imagens):
    """
    Função para buscar o nome do arquivo PDF associado a um gabarito_id
    a partir dos arquivos na pasta de imagens.
    """

    for file_name in os.listdir(pasta_imagens):
        if file_name.endswith('.jpeg'):

            gabarito_id = file_name.split('_')[1]



            with open(caminho_processed_files, 'r') as file:
                for linha in file:
                    partes = linha.strip().split('|')
                    if len(partes) >= 2:
                        nome_arquivo_pdf = partes[0].strip()
                        id_gabarito = partes[1].strip()

                        if id_gabarito == f'Gabarito_{gabarito_id}':

                            return nome_arquivo_pdf, gabarito_id

    raise ValueError("Gabarito não encontrado nos arquivos de imagens ou no 'processed_files.txt'.")

def criar_json_de_csv(caminho_csv, caminho_processed_files, pasta_imagens):
    """
    Função para criar um arquivo JSON a partir do CSV, associando um nome
    de arquivo obtido automaticamente a partir do 'processed_files.txt' e do nome do arquivo JPEG.
    """

    df = pd.read_csv(caminho_csv, sep=';')


    nome_arquivo, gabarito_id = obter_nome_arquivo_automatico(caminho_processed_files, pasta_imagens)


    id_criptografado = hashlib.sha256(nome_arquivo.encode()).hexdigest()


    json_final = {
        "id": id_criptografado,
        "nome_arquivo": nome_arquivo,
        "alunos": []
    }


    matriculas = df['MATRICULA'].unique()
    for matricula in matriculas:
        aluno_info = {
            "matricula": str(matricula),
            "questoes": {}
        }

        df_aluno = df[df['MATRICULA'] == matricula]
        for _, row in df_aluno.iterrows():
            questao = str(row['QUESTAO'])
            aluno_info["questoes"][questao] = {
                "A": int(row['A']),
                "B": int(row['B']),
                "C": int(row['C']),
                "D": int(row['D']),
                "E": int(row['E'])
            }

        json_final["alunos"].append(aluno_info)

    json_path = os.path.splitext(caminho_csv)[0] + ".json"
    with open(json_path, 'w') as json_file:
        json.dump(json_final, json_file, indent=4)

    print(f"JSON salvo em: {json_path}")

caminho_csv = '/content/matricula_e_respostas.csv'
caminho_processed_files = '/content/processed_files.txt'
pasta_imagens = '/content/images_JPEG'

criar_json_de_csv(caminho_csv, caminho_processed_files, pasta_imagens)